<a href="https://colab.research.google.com/github/beimnet777/Segmentation_model/blob/main/Coffee_leaf_disease_segmention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import cv2
import os

In [ ]:
from google.colab.patches import cv2_imshow

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
os.environ["SM_FRAMEWORK"] = "tf.keras"

In [ ]:
!pip install -U -q segmentation-models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 2.1 MB/s eta 0:00:00


In [ ]:
import segmentation_models as sm

Segmentation Models: using `tf.keras` framework.


In [ ]:
def mask_preparation(image):

  lower_red = np.array([0, 0, 128])
  upper_red = np.array([0, 0, 255])

  # Create a mask for the red color
  mask = cv2.inRange(image, lower_red, upper_red)

  image[mask != 0] = [255, 255, 255]
  image[mask == 0] = [0, 0, 0]

  red = image[:,:,2]

  return red

In [ ]:
def prepare_dataset(mask_path,image_path):
  images = []
  masks = [] 
        
  for root,dirs,files in os.walk(mask_path):
      for file in files:
          image_path_=os.path.join(image_path,file.split("_")[0]+".jpg")
          img = cv2.imread(image_path_)  

          img = cv2.resize(img, (512, 256))
          img=img.astype(np.float32)
          img/=255
          images.append(img)

          mask_path_=os.path.join(mask_path,file)
          mask = cv2.imread(mask_path_)

          mask = cv2.resize(mask, (512,256))
          # cv2_imshow(mask)
          mask=mask_preparation(mask)
          # cv2_imshow(mask)
          mask=np.expand_dims(mask,axis=-1)
          mask//=255
          # print(np.max(mask),np.min(mask))
          masks.append(mask.astype(np.float32))   
  masks = np.array(masks)
  images = np.array(images)
  return images,masks

In [10]:
train=prepare_dataset("/content/drive/MyDrive/annotations/train","/content/drive/MyDrive/images/train")

In [9]:
cv2_imshow(train[1][10]*255)

In [ ]:
train[1].shape

(400, 256, 512, 1)

In [ ]:
n_classes=2
activation='sigmoid'

dice_loss = sm.losses.DiceLoss() 
focal_loss = sm.losses.CategoricalFocalLoss()
total_loss = dice_loss + (1 * focal_loss)

metrics = [sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]

In [8]:
BACKBONE1 = 'resnet34'
preprocess_input1 = sm.get_preprocessing(BACKBONE1)
X_train = preprocess_input1(train[0])
y_train= train[1]

model1 = sm.Unet(BACKBONE1, encoder_weights='imagenet', classes=n_classes, activation=activation)

model1.compile(optimizer="sgd", loss=total_loss, metrics=metrics)
model1.summary()

In [ ]:
def create_sample_weight(y):
    sample_weight = np.ones_like(train[1])
    sample_weight[train[1] == 1] = 1.2
    return sample_weight

# create sample weights using tf.numpy_function
sample_weight = tf.numpy_function(create_sample_weight, [train[1]], tf.float64)

In [7]:
validation=prepare_dataset("/content/drive/MyDrive/annotations/val","/content/drive/MyDrive/images/val")

In [6]:
history1=model1.fit(X_train, y_train,batch_size=8, epochs=40,validation_data=validation,sample_weight=sample_weight)

In [ ]:
model1.save("/content/drive/MyDrive/coffee_fist_model_to_tune.h5")

In [5]:
test=prepare_dataset("/content/drive/MyDrive/annotations/test","/content/drive/MyDrive/images/test")


In [ ]:
prediction=model1.predict(test[0])

2/2 [==============================] - 12s 4s/step


In [ ]:
prediction=np.amax(prediction,axis=-1)

In [ ]:
prediction[prediction>=.5]=1
prediction[prediction<.5]=0

In [4]:
for index,val in enumerate(prediction):
  cv2_imshow(val*255)
  print("^^^^^^^^^^^^")
  cv2_imshow(test[1][index]*255)
  print("*************")

In [ ]:
def create_unet(width,height):
  WIDTH=width
  HEIGHT=height

  input=tf.keras.layers.Input((HEIGHT,WIDTH,3))
  # x=tf.keras.layers.Lambda(lambda y: y/255)(input)
  DROP_OUT=0.2

  c1=tf.keras.layers.Conv2D(16,3,activation="relu",padding="same")(input)
  d1=tf.keras.layers.Dropout(DROP_OUT)(c1)
  c2=tf.keras.layers.Conv2D(16,3,activation="relu",padding="same")(d1)
  p1=tf.keras.layers.MaxPooling2D((2,2))(c2)

  c3=tf.keras.layers.Conv2D(32,3,activation="relu",padding="same")(p1)
  d2=tf.keras.layers.Dropout(DROP_OUT)(c3)
  c4=tf.keras.layers.Conv2D(32,3,activation="relu",padding="same")(d2)
  p2=tf.keras.layers.MaxPooling2D((2,2))(c4)

  c5=tf.keras.layers.Conv2D(64,3,activation="relu",padding="same")(p2)
  d3=tf.keras.layers.Dropout(DROP_OUT)(c5)
  c6=tf.keras.layers.Conv2D(64,3,activation="relu",padding="same")(d3)
  p3=tf.keras.layers.MaxPooling2D((2,2))(c6)

  c7=tf.keras.layers.Conv2D(128,3,activation="relu",padding="same")(p3)
  d4=tf.keras.layers.Dropout(DROP_OUT)(c7)
  c8=tf.keras.layers.Conv2D(128,3,activation="relu",padding="same")(d4)
  p4=tf.keras.layers.MaxPooling2D((2,2))(c8)

  c9=tf.keras.layers.Conv2D(256,3,activation="relu",padding="same")(p4)
  d5=tf.keras.layers.Dropout(DROP_OUT)(c9)
  c10=tf.keras.layers.Conv2D(256,3,activation="relu",padding="same")(d5)
  u5=tf.keras.layers.Conv2DTranspose(128,(2,2),strides=(2,2))(c10)
  u5=tf.keras.layers.Concatenate()([u5,c8])



  c11=tf.keras.layers.Conv2D(128,3,activation="relu",padding="same")(u5)
  d6=tf.keras.layers.Dropout(DROP_OUT)(c11)
  c12=tf.keras.layers.Conv2D(128,3,activation="relu",padding="same")(d6)
  u6=tf.keras.layers.Conv2DTranspose(64,(2,2),strides=(2,2))(c12)
  u6=tf.keras.layers.Concatenate()([u6,c6])

  c13=tf.keras.layers.Conv2D(64,3,activation="relu",padding="same")(u6)
  d7=tf.keras.layers.Dropout(DROP_OUT)(c13)
  c14=tf.keras.layers.Conv2D(64,3,activation="relu",padding="same")(d7)
  u7=tf.keras.layers.Conv2DTranspose(32,(2,2),strides=(2,2))(c14)
  u7=tf.keras.layers.Concatenate()([u7,c4])

  c15=tf.keras.layers.Conv2D(32,3,activation="relu",padding="same")(u7)
  d8=tf.keras.layers.Dropout(DROP_OUT)(c15)
  c16=tf.keras.layers.Conv2D(32,3,activation="relu",padding="same")(d8)
  u8=tf.keras.layers.Conv2DTranspose(16,(2,2),strides=(2,2))(c16)
  u8=tf.keras.layers.Concatenate()([u8,c2])

  c17=tf.keras.layers.Conv2D(16,3,activation="relu",padding="same")(u8)
  d9=tf.keras.layers.Dropout(DROP_OUT)(c17)
  c18=tf.keras.layers.Conv2D(16,3,activation="relu",padding="same")(d9)


  outputs=tf.keras.layers.Conv1D(1,1,activation="sigmoid")(c18)

  model=tf.keras.Model(inputs=[input],outputs=[outputs])
  model.summary()
  return model

In [3]:
model=create_unet(512,256)

In [ ]:
model.compile(optimizer="sgd", loss=total_loss, metrics=metrics)

In [1]:
history=model.fit(train[0], train[1],batch_size=8, epochs=100,validation_data=validation)

In [ ]:
model.save("/content/drive/MyDrive/light_coffee_mode.h5")

In [ ]:
prediction=np.amax(prediction,axis=-1)

In [ ]:
prediction[prediction>=.5]=1
prediction[prediction<.5]=0

In [2]:
for index,val in enumerate(prediction):
  cv2_imshow(val*255)
  print("^^^^^^^^^^^^")
  cv2_imshow(test[1][index]*255)
  print("*************")